The Bickley jet is an idealized model of geophysical flows such as the Gulf Stream  and models a meandering zonal jet between a series of counter rotating vortices. In the reference frame moving with $ c_2 $, the flow consists of a steady background and a
time-periodic perturbation:

\begin{equation}
\mathbf{v}(x, y, t) = \begin{pmatrix} -\partial_y \psi(x, y, t) \\ \partial_x \psi(x, y, t) \end{pmatrix}, 
\end{equation}

where

\begin{align*}
\psi(x, y, t) &= \psi_0(x, y) + \psi_1(x, y, t) \\
\psi_0(x, y) &= c_2y-UL_y \tanh(\dfrac{y}{L_y}) \\
\psi_1(x,y,t) &= U L_ y\mathrm{sech}^2(\dfrac{y}{L_y}) [\epsilon_1 \cos(k_1x-\sigma_1t) + \epsilon_2 \cos(k_2x)]
\end{align*}

We have chosen the following set of parameters $$ c_1 = 0.205U, c_2 = 0.461U, \epsilon_1 = 0.15, \epsilon_2 = 0.3, U = 62.66 (\dfrac{m}{s}), L_y = 1.77\times 10^3 (km), L_x = 6371\pi(km), \sigma_1 = k_1(c_1-c_2) $$

The periodic period is given by:

\begin{equation}
T_{periodic} = \dfrac{2\pi}{\sigma_1}
\end{equation}

The velocity field has been discretized both in time and space and the velocity data is given over a gridded spatio-temporal domain.

In [1]:
# Import scipy
import scipy.io

# Import numpy
import numpy as np

In [2]:
def k(n, Len_X):
    
    return 2*n*np.pi/Len_X

def psi0(params, y):
    
    U = params[0]
    Len_Y = params[3]
    c2 = 0.461*U
    
    return -U*Len_Y*np.tanh(y/Len_Y)+c2*y

def psi1(params, x, y, t):
    
    U = params[0]
    R = params[1]
    Len_X = params[2]
    len_Y = params[3]
    eps1 = params[4][0]
    eps2 = params[4][1]

    c1 = 0.205*U
    c2 = 0.461*U
    sigma1 = k(1, Len_X)*(c1-c2)
    
    forcing2 = np.cos(k(2, Len_X)*x)
    forcing1 = np.cos(k(1, Len_X)*x-sigma1*t)
    
    return U*Len_Y*(sech(y/Len_Y)**2)*(eps1*forcing1 + eps2*forcing2)

def sech(x):
    
    return 2*np.exp(x)/(np.exp(2*x)+1)

def tanh(x):
    
    return (np.exp(2*x)-1)/(np.exp(2*x)+1)

def psi(params, x, y, t):

    return psi0(params, y)+psi1(params, x, y, t)

def Bickley_jet(params, x, y, t, dx, dy):
    
    X, Y, time = np.meshgrid(x, y, t)
    
    u = np.zeros(X.shape)
    v = np.zeros(Y.shape)
    
    for i in range(X.shape[0]):
        
        for j in range(Y.shape[1]):
            
            for r in range(time.shape[2]):
                
                x_ = X[i, j, r]
                y_ = Y[i, j, r]
                t_ = time[i, j, r]
                
                u[i, j, r] = -(psi(params, x_, y_ + dy, t_)-psi(params, x_, y_- dy, t_))/(2*dy)
                v[i, j, r] = (psi(params, x_ + dx, y_, t_)-psi(params, x_ - dx, y_, t_))/(2*dx)
    
    return u, v

In [3]:
# U = 62.66*3600*24 (in kilometers/day)
U = 62.66*3.6*24

# Radius of the Earth (in kms)
earthRadius = 6371

# L_x = pi*earthRadius
Len_X = np.pi*earthRadius

# L_y = 1.77*10**3 (in km)
Len_Y = 1.77*(10**3)

# [epsilon_1, epsilon_2]
epsilon = [.15, .3]

c1 = 0.205*U
c2 = 0.461*U
sigma1 = k(1, Len_X)*(c1-c2)

T_periodic = abs(2*np.pi/(sigma1))

# time array
t = np.linspace(0, T_periodic, 20, endpoint = True)
x = np.arange(-10*Len_X, 10*Len_X, 200)
y = np.arange(-3*Len_Y*1.25, 3*Len_Y*1.25, 200)

params = [U, earthRadius, Len_X, Len_Y, epsilon]

# Define grid-spacing for spatial derivative
dx = Len_X/5000
dy = Len_Y/5000

u, v = Bickley_jet(params, x, y, t, dx, dy)
            
scipy.io.savemat('./Bickley.mat', {'u': u, 'v': v, 'x': x, 'y': y, 't': t})

# References

[1] Rypina, I. I., Brown, M. G., Beron-Vera, F. J., Koçak, H., Olascoaga, M. J., & Udovydchenkov, I. A. (2007). On the Lagrangian dynamics of atmospheric zonal jets and the permeability of the stratospheric polar vortex. Journal of the Atmospheric Sciences, 64(10), 3595-3610.

[2] del‐Castillo‐Negrete, D., & Morrison, P. J. (1993). Chaotic transport by Rossby waves in shear flow. Physics of Fluids A: Fluid Dynamics, 5(4), 948-965.